# Hyperparameter Tuning using HyperDrive

Importing Dependencies

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

import joblib

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R75GZJZQW to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-140338
aml-quickstarts-140338
southcentralus
a0a76bad-11a1-4a2d-9887-97a29122c8ed


In [3]:
# Create compute cluster and choose a name for it
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-13T12:54:43.699000+00:00', 'errors': None, 'creationTime': '2021-03-13T12:52:30.085846+00:00', 'modifiedTime': '2021-03-13T12:52:45.533988+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 6, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


## Dataset


In [4]:
data = pd.read_csv('./heart_failure_clinical_records_dataset.csv')

found = False
key = "heart-failure-prediction"
description_text = "Prediction of patients’ survival with heart failure - Capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        my_dataset = 'https://raw.githubusercontent.com/gaya3arul/nd00333-capstone/master/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(my_dataset)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
                                
# Preview of the first five rows
print(data.head())

# Explore data
print(data.describe())

df = dataset.to_pandas_dataframe()
df.describe()

# Data columns
df.columns = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time', 'DEATH_EVENT']
x = df[['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']]
y = df[['DEATH_EVENT']]



    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.7           116    0   

   smoking  time  DEATH_EVENT  
0        0     4            1  
1        0     6            1  
2       

## Hyperdrive Configuration

For this experiment I am using a custom Scikit-learn Logistic Regression model, whose hyperparameters I am optimising using HyperDrive.

Early stopping policy

An early stopping policy is used to automatically terminate poorly performing runs thus improving computational efficiency. I chose the BanditPolicy which I specified as follows:

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

where:

evaluation_interval: This is optional and represents the frequency for applying the policy. Each time the training script logs the primary metric counts as one interval.

slack_factor: The amount of slack allowed with respect to the best performing training run. This factor specifies the slack as a ratio.

Any run that doesn't fall within the slack factor or slack amount of the evaluation metric with respect to the best performing run will be terminated. This means that with this policy, the best performing runs will execute until they finish and this is the reason I chose it.

Parameter Sampler
I specify the parameter sampler using the parameters C and max_iter. I chose discrete values with choice for both parameters.

C is the Regularization while max_iter is the maximum number of iterations.

RandomParameterSampling is one of the choices available for the sampler and I chose it because it is the faster and supports early termination of low-performance runs. If budget is not an issue, it would be better to use GridParameterSampling to exhaustively search over the search space or BayesianParameterSampling to explore the hyperparameter space.

## Parameter Sampler

I specify the parameter sampler using the parameters C and max_iter. I chose discrete values with choice for both parameters.

C is the Regularization while max_iter is the maximum number of iterations.

RandomParameterSampling is one of the choices available for the sampler and I chose it because it is the faster and supports early termination of low-performance runs. If budget is not an issue, it would be better to use GridParameterSampling to exhaustively search over the search space or BayesianParameterSampling to explore the hyperparameter space.

## HyperDriveConfig

The configuration chosen is as follows:

hyperparameter_sampling

The hyperparameter sampling space as defined above.

primary_metric_name

The name of the primary metric reported by the experiment runs. In our case, it is Accuracy.

primary_metric_goal

I chose PrimaryMetricGoal.MAXIMIZE. This parameter determines that the primary metric is to be maximized when evaluating runs.

policy

It refers to the early termination policy that is specified above.

estimator

An estimator that will be called with sampled hyperparameters. In this case, I choose estimator while the other two options are run_config and pipeline. The estimator will be used with train.py file which does a very basic manipulation of the data.

max_total_runs=16

The maximum total number of runs to create. This is the upper bound; there may be fewer runs when the sample space is smaller than this value. If both max_total_runs and max_duration_minutes are specified, the hyperparameter tuning experiment terminates when the first of these two thresholds is reached.

max_concurrent_runs=4

The maximum number of runs to execute concurrently. If None, all runs are launched in parallel. The number of concurrent runs is gated on the resources available in the specified compute target. Hence, you need to ensure that the compute target has the available resources for the desired concurrency.


In [5]:
experiment_name = 'heart-failure-prediction'
project_folder = './capstone-project'

experiment = Experiment(ws, experiment_name)
experiment

run = experiment.start_logging()

In [6]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Specify parameter sampler 
# Create the different params that you will be using during training

ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",
            compute_target=cpu_cluster_name,
            vm_size='STANDARD_DS3_V2',
            entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


## Run Details



In [7]:
# Submit the hyperdrive run to the experiment and show run details with the widget.

# Start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

# Monitor HyperDrive runs 
# You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba
Web View: https://ml.azure.com/experiments/heart-failure-prediction/runs/HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-140338/workspaces/quick-starts-ws-140338

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-13T13:01:43.022544][API][INFO]Experiment created<END>\n""<START>[2021-03-13T13:01:43.842418][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-03-13T13:01:44.409585][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-03-13T13:01:44.8587623Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba
Web View: https://ml.azure.com/experiments/heart-failure-prediction/runs/HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba?wsid=/s

In [8]:

# get_children_sorted_by_primary_metric: Returns a list of children sorted by their best primary metric.
# Each child in the result has run id, hyperparameters, best primary metric value and status.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))
print('===================================================')

# get_hyperparameters
# Return the hyperparameters for all the child runs that were launched by this HyperDriveRun.

print(hyperdrive_run.get_hyperparameters())
print('===================================================')

[{'run_id': 'HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba_10', 'hyperparameters': '{"--C": 500, "--max_iter": 50}', 'best_primary_metric': 0.8333333333333334, 'status': 'Completed'}, {'run_id': 'HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba_4', 'hyperparameters': '{"--C": 1, "--max_iter": 50}', 'best_primary_metric': 0.8333333333333334, 'status': 'Completed'}, {'run_id': 'HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba_0', 'hyperparameters': '{"--C": 500, "--max_iter": 200}', 'best_primary_metric': 0.8333333333333334, 'status': 'Completed'}, {'run_id': 'HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba_2', 'hyperparameters': '{"--C": 1, "--max_iter": 200}', 'best_primary_metric': 0.8333333333333334, 'status': 'Completed'}, {'run_id': 'HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba_3', 'hyperparameters': '{"--C": 100, "--max_iter": 300}', 'best_primary_metric': 0.8333333333333334, 'status': 'Completed'}, {'run_id': 'HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba_14', 'hyperparameters': '{"--C": 1000, "--max_iter": 100}', 'best_p

## Best Model


In [9]:
# Get the best run and save the model from that run.

# get_best_run_by_primary_metric()
# Returns the best Run, or None if no child has the primary metric.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())
print('===================================================')

# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
print('===================================================')

# get_file_names()
# Returns a list of the files that are stored in association with the run.

print("Best run file names :",best_run.get_file_names())
print('===================================================')

Best run metrics : {'Regularization Strength:': 100.0, 'Max iterations:': 300, 'Accuracy': 0.8333333333333334}
Best run details : {'runId': 'HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba_3', 'target': 'cpu-cluster', 'status': 'Completed', 'startTimeUtc': '2021-03-13T13:08:24.92226Z', 'endTimeUtc': '2021-03-13T13:09:02.88818Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '402ca2e6-e19c-4b40-a4ef-ac107c81e8ac', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '100', '--max_iter', '300'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'cpu-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'priority': None, 'credentialPassthrough': False, 

# Register the model

In [10]:

# Save the best model

best_run.register_model(model_name = "best_run_hyperdrive.pkl", model_path = './outputs/')

print(best_run)

Run(Experiment: heart-failure-prediction,
Id: HD_68a8df5a-aa2b-44a2-99d3-82e13768f3ba_3,
Type: azureml.scriptrun,
Status: Completed)


In [11]:
# Download the model file

best_run.download_file('outputs/model.pkl', 'hyperdrive_model.pkl')